In [2]:
from py_pdf_parser.loaders import load_file

document = load_file("../Malo_Adler_Thesis.pdf")
text2 = ""
for element in document.elements:
    text2 += element.text()


No elements detected on page 2, skipping this page.


In [3]:

import os
OPENAI_API_BASE= os.environ.get("OPENAI_API_BASE", "https://api-ai.numerique-interieur.com/v1") 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=OPENAI_API_KEY, openai_api_base=OPENAI_API_BASE, temperature=0)

In [4]:
from langchain_core.runnables import chain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from textrank import build_text_prompt, Model

openai_ef = OpenAIEmbeddingFunction(
    api_base="https://text-embeddings.c0.cloud-pi-native.com/",
    api_key="tom evil"
)

embedding_model = Model(openai_ef, "openai_ef")

prompt1 = """Write a summary in french of the following: 
{text}
SUMMARY :"""
prompt_template = PromptTemplate.from_template(prompt1)
output_parser = StrOutputParser()

@chain
def custom_chain(text):
    extractive_summary = build_text_prompt(text, 3_000, embedding_model)
    prompt_val = prompt_template.invoke(extractive_summary)
    output = llm.invoke(prompt_val)
    return output_parser.invoke(output)

In [5]:
len(openai_ef(["je voudrais", "du chocholat"]))

2

In [6]:
res = custom_chain.invoke(text2)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
res

"Le projet de licence en informatique et ingénierie comporte 30 crédits et porte sur l'automatisation du processus de résumé de textes judiciaires en français. L'objectif est d'utiliser des modèles de langage pré-entraînés pour extraire les informations pertinentes à partir des rapports auditionnels, qui peuvent être très longs et prendre beaucoup de temps à lire. Les données utilisées sont spécifiques et concernent le domaine judiciaire en France. Le projet sera évalué à la fois en termes de contenu et de fiabilité."